## Import model

In [131]:
## import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

CUDA = torch.cuda.is_available()
# device = "cpu"
device = torch.device("cuda" if CUDA else "cpu")

In [132]:
torch.manual_seed(42)

## Prep dataset

In [133]:
import sys
# run the below line once only
if "..\\chexnet" not in sys.path:
    sys.path.insert(0,r'..\chexnet')
print(sys.path)

['..\\chexnet', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [134]:
from DatasetGenerator import DatasetGenerator

In [135]:
pathDirData = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\raw_data\archive'
pathFileTrain = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt'
pathFileVal = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt'

transResize = 256
transCrop = 224
trBatchSize = 32
num_class = 14

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

torch.manual_seed(42)

transformList = []
transformList.append(transforms.Resize(transResize))
transformList.append(transforms.RandomResizedCrop(transCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

datasetTrain = DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileTrain, transform=transformSequence)
datasetVal =   DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileVal, transform=transformSequence)
train_loader = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=12, pin_memory=True)
val_loader = DataLoader(dataset=datasetVal, batch_size=trBatchSize, shuffle=False, num_workers=12, pin_memory=True)

Collected 2048 images from C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt
Collected 2048 images from C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt


## Create your own DenseNet

In [136]:
class dense_layer(nn.Module):
    def __init__(self, dim, training):
        super(dense_layer, self).__init__()
        eps = 1e-5
        momentum = 0.1
        hidden_dim = 128
        output_dim = 32
        self.training = training
        self.net = nn.Sequential(
            nn.BatchNorm2d(num_features=dim, eps=eps, momentum=momentum, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=dim, out_channels=hidden_dim, kernel_size=(1,1), stride=(1,1),bias=False),
            nn.BatchNorm2d(num_features=hidden_dim, eps=eps, momentum=momentum, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=hidden_dim, out_channels=output_dim, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
        )
        
    def forward(self, x):
        dropout_rate = 0
        return F.dropout(self.net(x), p = dropout_rate, training=self.training)     

In [137]:
# class _DenseLayer(nn.Module):
#     def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
#         super(_DenseLayer, self).__init__()
#         self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
#         self.add_module('relu1', nn.ReLU(inplace=True)),
#         self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
#                                            growth_rate, kernel_size=1, stride=1,
#                                            bias=False)),
#         self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
#         self.add_module('relu2', nn.ReLU(inplace=True)),
#         self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
#                                            kernel_size=3, stride=1, padding=1,
#                                            bias=False)),
#         self.drop_rate = float(drop_rate)
#         self.memory_efficient = memory_efficient

#     def bn_function(self, inputs):
#         "Bottleneck function"
#         # type: (List[Tensor]) -> Tensor
#         concated_features = torch.cat(inputs, 1)
#         bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
#         return bottleneck_output

#     def forward(self, input):  # noqa: F811
#         if isinstance(input, torch.Tensor):
#             prev_features = [input]
#         else:
#             prev_features = input

#         bottleneck_output = self.bn_function(prev_features)
#         new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
#         if self.drop_rate > 0:
#             new_features = F.dropout(new_features, p=self.drop_rate,
#                                      training=self.training)
#         return new_features

In [138]:
# class dense_layer(nn.Module):
#     def __init__(self, dim, training):
#         super(dense_layer, self).__init__()
#         eps = 1e-5
#         momentum = 0.1
#         hidden_dim = 128
#         output_dim = 32
#         self.training = training
#         # self.net = nn.Sequential(
#         self.norm1 = nn.BatchNorm2d(num_features=dim, eps=eps, momentum=momentum, affine=True, track_running_stats=True)
#         self.relu1 = nn.ReLU(inplace=True)
#         self.conv1 = nn.Conv2d(in_channels=dim, out_channels=hidden_dim, kernel_size=(1,1), stride=(1,1),bias=False)
#         self.norm2 = nn.BatchNorm2d(num_features=hidden_dim, eps=eps, momentum=momentum, affine=True, track_running_stats=True)
#         self.relu2 = nn.ReLU(inplace=True)
#         self.conv2 = nn.Conv2d(in_channels=hidden_dim, out_channels=output_dim, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
#         # )
        
#     def forward(self, x):
#         dropout_rate = 0
#         x = self.norm1(x)
#         x = self.relu1(x)
#         x = self.conv1(x)
#         x = self.norm2(x)
#         x = self.relu2(x)
#         x = self.conv2(x)
#         return F.dropout(x, p = dropout_rate, training=self.training)     

In [139]:
class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs):
        "Bottleneck function"
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output

    def forward(self, input):  # noqa: F811
        if isinstance(input, torch.Tensor):
            prev_features = [input]
        else:
            prev_features = input

        bottleneck_output = self.bn_function(prev_features)
        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features 

In [140]:
class dense_block(nn.ModuleDict):
    def __init__(self, layer_count, dim, training):
        super(dense_block, self).__init__()
        hidden_dim = dim
        growth_rate = 32
        for layer_index in range(layer_count):
            # setattr(self, f'layer_{layer_index}',dense_layer(hidden_dim, training=training))
            # ## update the hidden_dim so that the input size of the next layer has space
            # ## for all the outputs that precede it
            # hidden_dim += getattr(self,f'layer_{layer_index}').net[5].out_channels
            layer = dense_layer(
                dim = hidden_dim,
                training=training
            )
            self.add_module(f"dense_layer{layer_index}", layer)
            hidden_dim += growth_rate


    def forward(self, input):
        output = input
        for _, layer in self.items():
            # print("working in " + str(layer._get_name()) + " ...")
            new_output = layer(output)
            # print("output of this dense_layer is " + str(new_output.shape))
            ## add the new_output to the previous output
            ## for the next layer in the block
            # print("output shape", str(output.shape))
            # print("new_output shape", str(new_output.shape))
            output = torch.cat((output,new_output),1) 
        return output

In [141]:
# class dense_block(nn.ModuleDict):
#     def __init__(self, layer_count, dim, training):
#         super(dense_block, self).__init__()
#         hidden_dim = dim
#         growth_rate = 32
#         for layer_index in range(layer_count):
#             # setattr(self, f'layer_{layer_index}',dense_layer(hidden_dim, training=training))
#             # ## update the hidden_dim so that the input size of the next layer has space
#             # ## for all the outputs that precede it
#             # hidden_dim += getattr(self,f'layer_{layer_index}').net[5].out_channels
#             layer = _DenseLayer(
#                 num_input_features = dim + layer_index * growth_rate,
#                 growth_rate=growth_rate,
#                 bn_size=4,
#                 drop_rate=0,
#                 memory_efficient=False,
#             )
#             self.add_module(f"dense_layer{layer_index}", layer)
#             hidden_dim += growth_rate


#     def forward(self, input):
#         output = input
#         for _, layer in self.items():
#             # print("working in " + str(layer._get_name()) + " ...")
#             new_output = layer(output)
#             # print("output of this dense_layer is " + str(new_output.shape))
#             ## add the new_output to the previous output
#             ## for the next layer in the block
#             # print("output shape", str(output.shape))
#             # print("new_output shape", str(new_output.shape))
#             output = torch.cat((output,new_output),1) 
#         return output

In [142]:
class transition_block(nn.Module):
    def __init__(self, channels):
        super(transition_block, self).__init__()
        eps = 1e-5
        momentum = 0.1
        self.norm = nn.BatchNorm2d(channels, eps=eps, momentum=momentum, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(channels, channels//2, kernel_size=(1,1), stride=(1,1), bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2,stride=2,padding=0)

    def forward(self, input):
        x = input
        for layer in self.children():
            # print("working in " + str(layer._get_name()) + " ...")
            x = layer(x)
            # print("output of this transition is " + str(x.shape))

        return x


In [143]:
class dense_net(nn.Module):
    def __init__(self, num_class, training):
        super(dense_net, self).__init__()
        hidden_dim = 64

        self.initial_setup = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=hidden_dim, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False),
            nn.BatchNorm2d(num_features=hidden_dim, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride = 2,padding=1, dilation=1, ceil_mode=False)
        )

        ## just calculate the number of channels in trans blocks manually
        ## since we know initial channel number and how many layers are in each block
        self.denseblock1 = dense_block(layer_count=6, dim=hidden_dim, training=training)
        self.trans1 = transition_block(channels = hidden_dim + 6 * 32) # 256 
        self.denseblock2 = dense_block(layer_count=12, dim=hidden_dim * 2, training=training)
        self.trans2 = transition_block(channels = hidden_dim * 2 + 12 * 32) #512
        self.denseblock3 = dense_block(layer_count=24, dim=hidden_dim * 4, training=training)
        self.trans3 = transition_block(channels = hidden_dim * 4 + 24 * 32) # 1024
        self.denseblock4 = dense_block(layer_count=16, dim=hidden_dim * 8, training=training)
        self.norm5 = nn.BatchNorm2d(num_features=hidden_dim * 8 + 16 * 32)

        ## preparing input into the classifier
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1,1)) ## global average pooling
        self.flatten = nn.Flatten()

        ## classifier component:
        self.classifier = nn.Sequential(
            nn.Linear(in_features=self.norm5.num_features, out_features=num_class, bias=True),
            nn.Sigmoid()
        )

        ## initialize to random values
        kaiming = norm = linear = False
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                kaiming = True
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                norm = True
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
                linear = True

        print(kaiming,norm,linear)

    def forward(self, input):
        x = input
        for block in self.children():
            # print("working in " + str(block._get_name()) + " ...")
            x = block(x)
            # print("output of this block is " + str(x.shape))
        return x

In [144]:
model = dense_net(14, training = True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device)

True True True


dense_net(
  (initial_setup): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (denseblock1): dense_block(
    (dense_layer0): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (dense_layer1): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [145]:
model.initial_setup[0].weight

Parameter containing:
tensor([[[[ 0.2272, -0.0758, -0.0426,  ...,  0.0247, -0.0265,  0.0591],
          [-0.1045, -0.0152,  0.3049,  ...,  0.0035,  0.1152, -0.1008],
          [-0.0401, -0.0739, -0.0767,  ..., -0.0596, -0.1044, -0.1815],
          ...,
          [ 0.2348, -0.0762, -0.0697,  ...,  0.0192,  0.1652,  0.0346],
          [ 0.1162, -0.0160, -0.0116,  ..., -0.2688,  0.3540,  0.0842],
          [-0.0030, -0.0638,  0.0365,  ..., -0.0308,  0.1739, -0.0121]],

         [[-0.0104, -0.1915, -0.1581,  ..., -0.0985, -0.0422,  0.0477],
          [ 0.1287,  0.0841,  0.0459,  ..., -0.0032,  0.1101, -0.0705],
          [ 0.0070,  0.1659, -0.0404,  ..., -0.0164, -0.0870,  0.0342],
          ...,
          [-0.0533, -0.1078,  0.0498,  ..., -0.0314,  0.0680, -0.2729],
          [ 0.0389, -0.0120, -0.0760,  ..., -0.0830,  0.1380,  0.0346],
          [-0.2495, -0.1623, -0.0374,  ...,  0.2057, -0.0067,  0.0479]],

         [[-0.0412, -0.1994, -0.0258,  ...,  0.1540,  0.0191, -0.0733],
        

## Train Model

In [146]:
## function to calculate the F1 score
def f1_score(tp, fp, fn):
    return 2 * (tp) / (2 * tp + fp + fn)

In [147]:
# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 5, mode = 'min')

# Create a TensorBoard writer
writer = SummaryWriter()
model_name = "custom_dnet_conf_1_14_original_v5"

# Train the model
n_epochs = 5
lossMIN = 100000
model.train()
for epoch in range(n_epochs):

    for i, (images, labels, _) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        print(images.shape)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # calculate statistics
        # pred_labels = (nn.Softmax(dim=1)(outputs) > 1/14).long()
        
        tp_array = [0 for x in range(num_class)]
        fp_array = [0 for x in range(num_class)]
        fn_array = [0 for x in range(num_class)]
        pred_labels = (outputs > 1/14).long()
        tp_array += sum(torch.logical_and(pred_labels, labels))
        fp_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), pred_labels))
        fn_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), labels))
        
        writer.add_scalar('Loss/train', loss, epoch * len(train_loader) + i)
        writer.add_scalar('TP_Sum/train', sum(tp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FP_Sum/train', sum(fp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FN_Sum/train', sum(fn_array), epoch * len(train_loader) + i)
        writer.add_scalar('F1_Score/train', f1_score(sum(tp_array), sum(fp_array), sum(fn_array)), epoch * len(train_loader) + i)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)

        # Display
        # if (i + 1) % 100 == 0:
        print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, tp_sum: {:.4f}, fp_sum: {:.4f}, fn_sum: {:.4f}, cumulative_f1_score: {:.4f}".format(epoch + 1, \
                                                                     n_epochs, \
                                                                     i + 1, \
                                                                     len(train_loader), \
                                                                     loss,\
                                                                     sum(tp_array), \
                                                                     sum(fp_array),\
                                                                     sum(fn_array),\
                                                                     f1_score(sum(tp_array), sum(fp_array), sum(fn_array))))
        # print("outputs\n", outputs)
        # print("pred_labels\n", pred_labels)
        # print("actual labels\n", labels)

        if loss < lossMIN:
                lossMIN = loss    
                torch.save({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, r'./dnet_models/m-' + model_name + '.pth.tar')
                
    

torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [1/64], Loss: 0.6967, tp_sum: 25.0000, fp_sum: 423.0000, fn_sum: 0.0000, cumulative_f1_score: 0.1057
torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [2/64], Loss: 0.7703, tp_sum: 23.0000, fp_sum: 424.0000, fn_sum: 0.0000, cumulative_f1_score: 0.0979
torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [3/64], Loss: 0.7116, tp_sum: 24.0000, fp_sum: 423.0000, fn_sum: 0.0000, cumulative_f1_score: 0.1019
torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [4/64], Loss: 0.7082, tp_sum: 25.0000, fp_sum: 423.0000, fn_sum: 0.0000, cumulative_f1_score: 0.1057
torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [5/64], Loss: 0.7103, tp_sum: 24.0000, fp_sum: 424.0000, fn_sum: 0.0000, cumulative_f1_score: 0.1017
torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [6/64], Loss: 0.6999, tp_sum: 20.0000, fp_sum: 428.0000, fn_sum: 0.0000, cumulative_f1_score: 0.0855
torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [7/64], Loss: 0.6846, tp_sum: 29.0000, fp_sum: 419.0000, f

In [148]:
torch.save({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, r'./dnet_models/m-' + model_name + '.pth.tar')
            

In [149]:
model_name

'custom_dnet_conf_1_14_original_v5'

In [150]:
from sklearn.metrics import roc_auc_score
import numpy as np

def to_tensor(crops):
    return torch.stack([transforms.ToTensor()(crop) for crop in crops])

def normalize_crops(crops):
    normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    normalized_crops = []
    for crop in crops:
        normalized_crop = normalize(crop)  # Assuming normalize is a function you have defined
        normalized_crops.append(normalized_crop)
    return torch.stack(normalized_crops)

def computeAUROC (dataGT, dataPRED, classCount):
    
    outAUROC = []
    
    datanpGT = dataGT.cpu().numpy()
    datanpPRED = dataPRED.cpu().numpy()
    
    for i in range(classCount):
        outAUROC.append(roc_auc_score(datanpGT[:, i], datanpPRED[:, i]))
        
    return outAUROC

def test (pathDirData, pathFileTest, pathModel, nnClassCount, trBatchSize, transResize, transCrop):   
        
        CLASS_NAMES = [ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
                'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']
        
        model = dense_net(14, training=False)
        modelCheckpoint = torch.load(pathModel)
        model.load_state_dict(modelCheckpoint['state_dict'])
        model.to(device)

        #-------------------- SETTINGS: DATA TRANSFORMS, TEN CROPS
        normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        #-------------------- SETTINGS: DATASET BUILDERS
        transformList = []
        transformList.append(transforms.Resize(transResize))
        transformList.append(transforms.TenCrop(transCrop))

        transformList.append(to_tensor)
        # transformList.append(transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])))
        transformList.append(normalize_crops)
        # transformList.append(transforms.Lambda(lambda crops: torch.stack([normalize(crop) for crop in crops])))
        transformSequence=transforms.Compose(transformList)
        
        datasetTest = DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileTest, transform=transformSequence)
        dataLoaderTest = DataLoader(dataset=datasetTest, batch_size=trBatchSize, num_workers=0, shuffle=False, pin_memory=True)
        
        outGT = torch.FloatTensor().cuda()
        outPRED = torch.FloatTensor().cuda()
       
        model.eval()
        print('before enumeration begins')
        for i, (input, target, _) in enumerate(dataLoaderTest):
            # print(i)
            target = target.cuda()
            outGT = torch.cat((outGT, target), 0)
            
            bs, n_crops, c, h, w = input.size()
            with torch.no_grad():
                varInput = torch.autograd.Variable(input.view(-1, c, h, w).cuda())
            
                out = model(varInput)
                outMean = out.view(bs, n_crops, -1).mean(1)
                
                outPRED = torch.cat((outPRED, outMean.data), 0)

        aurocIndividual = computeAUROC(outGT, outPRED, nnClassCount)
        aurocMean = np.array(aurocIndividual).mean()
        
        print ('AUROC mean ', aurocMean)
        
        for i in range (0, len(aurocIndividual)):
            print (CLASS_NAMES[i], ' ', aurocIndividual[i])
        
     
        return


In [151]:
test (pathDirData = pathDirData, 
      pathFileTest = r'../chexnet/dataset/test_1.txt', 
      pathModel = f'./dnet_models/m-{model_name}.pth.tar', 
      nnClassCount = 14, 
      trBatchSize = trBatchSize, 
      transResize = transResize, 
      transCrop = transCrop)

True True True
Collected 2048 images from ../chexnet/dataset/test_1.txt
before enumeration begins
AUROC mean  0.5095133236789442
Atelectasis   0.6075298180088441
Cardiomegaly   0.41596002329930415
Effusion   0.4868496283590623
Infiltration   0.579702051402635
Mass   0.5511618939038294
Nodule   0.5636917286354667
Pneumonia   0.4135702135702136
Pneumothorax   0.44408713692946056
Consolidation   0.6018611670020121
Edema   0.5701375164690382
Emphysema   0.4491948063360942
Fibrosis   0.46562888768381766
Pleural_Thickening   0.5488754479329186
Hernia   0.4349362119725221
